### **Install SPLADE Index**

In [1]:
# ! pip install -Uq datasets huggingface_hub numba "jax[cpu]"

In [ ]:
! git clone https://github.com/rasyosef/splade-index.git
! pip install -q ./splade-index

### **Load Dataset**

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "philschmid/finanical-rag-embedding-dataset",
    split="train"
  ).shuffle(seed=42).select(range(1000))

dataset

In [4]:
documents = list(dataset["context"])
len(documents), documents[:3]

(1000,
 ["The company's debt obligations as of December 31, 2023, totaled $2,299,887 thousand.",
  "A corporate entity referred to as a management services organization (MSO) provides various management services and keeps the physician entity 'friendly' through a stock transfer restriction agreement and/or other relationships. The fees under the management services arrangement must comply with state fee splitting laws, which in some states may prohibit percentage-based fees.",
  'We manufacture and distribute our products through facilities in the United States (U.S.), including Puerto Rico, and in Europe and Asia.'])

### **Index the Documents**

In [ ]:
from sentence_transformers import SparseEncoder

# Download a SPLADE model from the 🤗 Hub
model = SparseEncoder("rasyosef/splade-tiny")

In [ ]:
from splade_index import SPLADE

# Create the SPLADE retriever and index the corpus
retriever = SPLADE()
retriever.index(model=model, documents=documents)

### **Query the Index**

In [7]:
# Query the corpus
queries = list(dataset["question"])
len(queries), queries[:3]

(1000,
 ["How much were the company's debt obligations as of December 31, 2023?",
  'What are the specific structures and legal considerations for a management services organization (MSO) in relation to its relationship with physician owners?',
  'Where does Eli Lilly and Company manufacture and distribute its products?'])

In [ ]:
# Get top-k results as a tuple of (doc ids, scores). Both are arrays of shape (n_queries, k).
results = retriever.retrieve(queries, k=5)
doc_ids, result_docs, scores = results.doc_ids, results.documents, results.scores

In [9]:
print("Query:", queries[0])

print("Retrieved Documents:")
for i in range(doc_ids.shape[1]):
    doc_id, doc, score = doc_ids[0, i], result_docs[0, i], scores[0, i]
    print(f"Rank {i+1} (score: {score:.2f}) (doc_idx: {doc_id}):", doc)

Query: How much were the company's debt obligations as of December 31, 2023?
Retrieved Documents:
Rank 1 (score: 29.81) (doc_idx: 0): The company's debt obligations as of December 31, 2023, totaled $2,299,887 thousand.
Rank 2 (score: 26.19) (doc_idx: 422): As of December 30, 2023, the Company's net deferred tax assets were $639,953, and as of December 31, 2022, they were $311,106.
Rank 3 (score: 25.50) (doc_idx: 798): As of December 31, 2023, the total invested assets at Humana Inc. were reported to be $21,590 million.
Rank 4 (score: 25.34) (doc_idx: 842): The company's finance lease obligations were totaled at $156,854 thousand as of December 31, 2023.
Rank 5 (score: 25.31) (doc_idx: 218): During the year ended December 30, 2023, the Company completed business acquisitions for a total consideration of $134 million that resulted in the recognition of $49 million of identifiable net assets and $85 million of goodwill.
